In [1]:
from IPython.core.display import HTML  #For a more pleasing rendering...
HTML(open("styles/custom.css").read()) 

# 0. Installation Guide

We will start with an installation guide for your own PC, if you also want to set up an server check the [bottom](#Setting-up-the-Server) page first. 

## OpenAi Gym

** "A toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Go." ** 

__[Homepage](https://gym.openai.com/)__ || __[GitHub](https://github.com/openai/gym)__  


We are using Ubuntu 16.04. with Python 2.7 already installed,
so in order to use all the environments in the OpenAI gym we need the following packages: 

```
$ sudo apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig 
```

After that we can use pip to do a full install of the gym.

```
$ sudo pip install gym[all] # Full Installation

$ sudo pip install gym # Minimal Installation see git
```

## Tensorflow

 **"An open-source software library for Machine Intelligence.
TensorFlow™ is an open source software library for numerical computation using data flow graphs." ** 

__[Homepage](https://www.tensorflow.org/)__ || __[GitHub](https://github.com/tensorflow/tensorflow)__  


Chose one of the following packages.
For other Python versions and a more detailed installation guide visit the homepage.

```
$ sudo pip install tensorflow      # Python 2.7; CPU support (no GPU)
$ sudo pip install tensorflow-gpu  # Python 2.7; GPU support

```

For further optimisation it is necessary to build the tensorflow source on your own. Which is explained [here](https://www.tensorflow.org/install/install_sources).

## Getting Jupyter

__[Homepage](http://jupyter.org/index.html)__ || __[Installation](http://jupyter.org/install.html)__  

If you want to follow the guide more closely we strongly recommend installing jupyter and run our notebooks with it.

```
$ sudo apt-get -y install ipython ipython-notebook
$ sudo pip install jupyter 
```

To run the notebook open your terminal write:

```ruby
$ jupyter notebook 
```

Now you can navigate to our notebooks and open them. 

If you are not familiar with jupyter don't worry, the layout is very intuitive and we will mention important things in our guide anyways. 

### Note

Whenever you get errors during the installation make sure that all your packages, especially pip are up to date. Also note that we are using Python 2.7. 

## Test your installation

by running the following code either in your installed jupyter notebook session or in your local python environment. (To run the code press Shift+Enter in the cell.) 

In [2]:
import gym
env = gym.make('Pong-v0') #for a full install
#env = gym.make('CartPole-v0') #for the minimal install
env.reset()
env.render()

[2017-09-22 10:25:34,705] Making new env: Pong-v0


you should now see a rendered image of the game environment. 

<font> Unfortunately to remove the window you have to restart the IPython Kernel. To do this simply klick on the Kernel tab above and choose Restart. Generally if you get any problems running the code restarting the kernel is the first thing I would suggest trying.  </font>

In [3]:
import tensorflow as tf

hello = tf.constant('Hello, Tensorflow!')
sess = tf.Session()
print(sess.run(hello))

a = tf.constant(10)
b = tf.constant(32)
print(sess.run(tf.add(a,b)))
sess.close()

Hello, Tensorflow!
42


Output: 

```Hello, Tensorflow!
42 ```

Now let's get started...

## Setting up the Server

### Adding users and permissions

We are using Ubuntu 16.04 as OS and only have only our root account that we logged in to via ssh. As always at first we add a new user to the system: 

```
$ sudo useradd -m newuser
```

Note that '-m' creates the home directory for newuser. In order to give him a password write: 

```
$ sudo passwd newuser
```

To create a group with sudo permission: 

```
$ sudo groupadd wheel 
```

Now we edit the sudoers file to give the group member the rights.

```
$ sudo visudo
```

Below  ```#Allow members group sudo...``` add: 

```
%wheel ALL=(ALL:ALL) ALL
```

Save the file. Make sure to remove the .tmp ending if there is one and that you overwrite the existing one correctly.

Now we want to add our newuser to our group wheel: 

```
$ usermod -a -G wheel newuser
```

Now we log in as our newuser and if we want block root access from now on. 

### Installing Remote Notebooks

### If you want multi user access on your server: 

### Jupyterhub

<font> Jupyterhub will be our choice to host multiple user notebooks. It provides an only login page that redirects users to ther own instance of an running jupyter notebook. The accounts will be the user accounts on our OS and the work path will be their home directory. Much more information can be found at their git [repository](https://github.com/jupyterhub/jupyterhub). At first we will install the dependencies and then jupyterhub itself: </font>

```
$ sudo apt-get install python3 
$ sudo apt-get install npm nodejs-legacy
$ npm install -g configurable-http-proxy
$ sudo apt-get -y install ipython ipython-notebook
$ sudo pip3 install jupyter
$ sudo pip3 install jupyterhub 
$ sudo pip3 install --upgrade notebook
$ sudo python2 -m pip install ipykernel
$ sudo python2 -m ipykernel install 
```

Now we choose a directory where we will start jupyterhub and store the configs for example ``` /etc/jupyterhub```. Go there and execute: 

In order to get a SSL protected connection; in the same directory we run: 

```
$ openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mykey.key -out mycert.pem
```

And then create a config for jupyterhub: 

```
$ jupyterhub --generate-config
```

Open up the ```jupyterhub_config.py``` and uncomment/edit the following lines: 

````
## The public facing ip of the whole application (the proxy)
c.JupyterHub.ip = '0.0.0.0' # Enter your server ip here. 

## The public facing port of the proxy
c.JupyterHub.port = 443 ## 443 for https, 80 for http
    ## Choose your server ip adress and port to allow public access.
    
    
## Use with ssl_key
c.JupyterHub.ssl_cert = 'mycert.pem'

## Use with ssl_cert
c.JupyterHub.ssl_key = 'mykey.key'
    ## Add your SSL key and cert for a more secure connection.
    
## Defaults to an empty set, in which case no user has admin access.
c.Authenticator.admin_users = {'newuser'}    
    ## Give your user admin rights for the browser control panel.
```

That's all. We will be using the default PAMAuthenticator and LocalProcessSpawner, which you can read more about [here](https://github.com/jupyterhub/jupyterhub/wiki/Authenticators). That means we log in with a user on our OS and the jupyter instance will be a new local process. 

<font>Make sure everything you want to use in the jupyter notebooks is installed for every user on the server! Also all the dependencies for the installed things have to be installed globally, to achieve this for example with python packages use: ``` sudo pip``` instead of only ```pip```.</font>

To start jupyterhub go to your choosen directory and simply write: 

```
$ jupyterhub
```

You should now be able to log in with your servers ip adress  https://0.0.0.0:443 and your user account. 

### If you want single user access on your server: 

Follow this [guide](http://jupyter-notebook.readthedocs.io/en/stable/public_server.html). You just install jupyter on your server the same way you would do it on your local PC, like [above](#Getting-Jupyter). Then you set a password and configure SSL and connection information like we did for jupyterhub. Finally you can connect on a single jupyter instance from your browser. 

### OpenAi Gym

This is exactly the same installation as [above](#OpenAi-Gym).

### Tensorflow

The same goes for [Tensorflow](#Tensorflow). Even though it's even more recommended to build it from source here. 

Of course you can test your installation as mentioned [above](#Test-your-installation ).

### Starting applications on your server

```
#to run the Jupyter Notebook

$ sudo jupyter notebook --port=443 --ip=176.9.5.89 --allow-root&


#to run the Tensorboard

$ sudo tensorboard --logdir 'path/to/your/logs' --port=80 --host 0.0.0.0&


#to run Jupyterhub

$ cd /etc/jupyterhub/
$ tmux
$ sudo jupyterhub
#ctrl+b d to detach window
# $ tmux #to attach it again

```

### Keeping users in sync with our git repository

<font>We have a git repository with our notebooks that we want to share with our users. So we created a technical user that has a crontab that automatically pulls the repo into a public folder. From there the users can copy it by themself. 
$$ $$
When we add a user with ``` "sudo adduser user"``` we also copy them a ```"Welcome.ipynb"``` into their home folder, which includes this cell: 
</font>

```
%%bash
cp -Rrf /home/tec/public/OpenAI_Gym/ ./
```

<font>  So as soon as the user executes it he will get the most recent version of the repository. In order to avoid getting his files overwritten the user has to change the second path for getting a newer version after it.  </font>